# 1. Set libraries

In [199]:
'''
!pip install sklearn
!pip install sympy
!pip install tabulate
!pip install arch
!pip install pymc3
!pip install pandas_datareader
!pip install wrds
!pip install xlrd
!pip install openpyxl
!pip install ipy_table
'''

'\n!pip install sklearn\n!pip install sympy\n!pip install tabulate\n!pip install yfinance\n!pip install yahoofinancials\n!pip install arch\n!pip install pymc3\n!pip install pandas_datareader\n!pip install wrds\n!pip install xlrd\n!pip install openpyxl\n!pip install ipy_table\n'

In [200]:
from lib import *
%matplotlib inline

# 2. Generalized factor model: model descriptions

## 2.1 CAPM 
The CAPM is based on the idea that investors are compensated for market exposure. In other words, CAPM suggests there is only one risk factor driving asset returns and it is the exposure to market returns (in excess of risk free rate).

The CAPM equation of asset risk premium can be written as it follows: 

$$
\begin{align*}
    R_{t}^{i}-R_{t}^{f}=\alpha _{i}+\beta _{i}\left( R_{t}^{M}-R_{t}^{f}\right)+\varepsilon _{t}^{i}.
\end{align*}
$$

where: 
- $R_{t}^{i}$ the return of the asset i at time t
- $R_{t}^{f}$ the risk free return at time t
- $R_{t}^{M}$ the market return at time t


*Reference: Ang, Asset managment: A systematic approach to factor investing*

## 2.2 Fama French five factor model

1. the market factor **MKT** as in the CAPM model
2. the size factor **SMB** measures the historic excess returns of small caps over big caps
3. the value factor **HML** measures the historic excess returns of value stocks over growth stocks
4. the profitability factor **RMW** measures the difference between the returns of firms with robust (high) and weak (low) operating profitability;
5. the investment factor **CMA** measures the difference between the returns of firms that invest conservatively and firms that invest aggressively.

**Remark** The Carhart four-factor model will be added as well

# 3. Data loading

- We will use the <code>Mkt-RF</code> and `RF` from **Ken French Data Library** respectively as the market excess return $R_{t}^{M}-R_{t}^{f}$ and risk-free rate $R_{t}^{f}$; 

- We will use **BBG** data for assets returns $R^{i}_t$

**Remark**: We focus on daily returns for now

In [177]:
## Period of data
start_date = dt.datetime(2015,12,31)
end_date = dt.datetime(2020,1,1)
ticker = 'TSLA'

## 3.1 Fama French factor returns data

In [102]:
## Load French-Fama daily factor returns
ff_factors = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily','famafrench', start=start_date)

In [103]:
## Save factor retunrs in dataframe and add date column to be used to merge 
ff5_full = ff_factors[0]
ff5_full['date'] = ff5_full.index
ff5_full

,Mkt-RF,SMB,HML,RMW,CMA,RF,date
Date,,,,,,,
2015-12-31,-0.920,-0.220,0.250,-0.120,0.090,0.000,2015-12-31
2016-01-04,-1.590,-0.730,0.550,0.360,0.430,0.000,2016-01-04
2016-01-05,0.120,-0.250,0.030,0.040,0.350,0.000,2016-01-05
2016-01-06,-1.350,-0.220,-0.030,0.140,0.040,0.000,2016-01-06
2016-01-07,-2.440,-0.270,0.060,0.510,0.430,0.000,2016-01-07
...,...,...,...,...,...,...,...
2020-12-24,0.220,-0.430,-0.170,0.240,-0.050,0.000,2020-12-24
2020-12-28,0.460,-0.650,0.320,1.460,0.500,0.000,2020-12-28
2020-12-29,-0.400,-1.420,0.240,0.750,-0.280,0.000,2020-12-29


In [104]:
#ff5_full.to_csv('Fama_French_FiveFactors_daily_returns')

## 3.2 BBG Asset returns

In [105]:
## Load BBG data: asset daily returns
asset_returns = pd.read_excel("Russell3000_ts.xlsx", 
                              sheet_name='One day return', 
                              engine='openpyxl', 
                              header=4, 
                              index_col=0, 
                              skipfooter=3)

In [106]:
asset_returns.index.name = 'Date'
asset_returns.head()
#asset_returns.tail()

,COKE UW Equity,CVI UN Equity,AA UN Equity,INO UW Equity,VZ UN Equity,SPWH UW Equity,SABR UW Equity,CAT UN Equity,JPM UN Equity,CVX UN Equity,...,AAN UN Equity,VNT UN Equity,MAXN UW Equity,IBEX UQ Equity,CMPI UQ Equity,FHTX UQ Equity,JAMF UW Equity,AZYO UQ Equity,NARI UW Equity,VRM UW Equity
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,2.090,3.048,0.570,1.525,0.385,0.956,-0.888,0.382,-0.144,0.357,...,-15.849,-2.941,-10.002,11.364,-6.714,-5.077,1.735,-6.621,6.093,4.363
2015-01-05,-0.746,-3.409,-5.793,-0.429,-0.831,-0.406,-1.543,-5.279,-3.104,-3.997,...,-13.946,-7.576,-10.003,-9.038,3.369,-6.977,-0.451,-6.942,-4.656,-8.782
2015-01-06,-0.684,-1.168,0.735,-0.539,1.009,-2.174,-1.416,-0.643,-2.593,-0.046,...,-4.065,1.639,16.147,-2.564,1.852,-5.500,1.486,-2.778,-0.581,-5.636
2015-01-07,0.610,-1.103,2.588,1.192,-1.807,4.028,-0.051,1.550,0.153,-0.083,...,-0.869,9.677,-49.495,-0.658,-1.818,5.820,0.521,0.490,2.924,-0.070
2015-01-08,1.178,5.019,2.846,0.214,2.143,-0.534,3.694,1.025,2.235,2.288,...,2.740,2.941,5.526,-2.914,2.889,-0.125,-0.494,-4.143,8.523,-2.093


# 4. Generic computation of the $\beta$s

In [108]:
def get_ff_beta(ticker, start=start_date, end=end_date, X = ['Mkt-RF'], print_results = True, return_results = False):
    '''
    ___DESCRIPTION___
    Returns the beta for specified ticker (regression results are available to display)
    
    ___INPUTS___
    - ticker
    - start corresponding to start date
    - end corresponding to end date
    - X the loading factor ie the risk factors, set to Market factor by default
    - print_results
    - return_results
    
    ___OUTPUTS___
    - OLS summary results
    - OLS
    
    ___TO BE ADDED LATER___
    - Possibility to set frequency of data
    - Possibility to perform robust regression
    '''
    
    # select columns with right ticker
    df = asset_returns[[col for col in asset_returns.columns if ticker in col]]
    if len(df.columns)>2: 
        print('WARNING \n Two columns with similar tickers found!')
    
    # select columns with right time period
    df = df[(df.index>=start) & (df.index<=end)]
    df['date'] = df.index
    
    # Build meta dataframe with loadings and observed data
    df_ff = df.merge(ff5_full, left_on='date', right_on='date', how = 'outer').dropna()
    df_ff.index = df_ff.date
    df_ff['Excess return'] = df_ff[df_ff.columns[0]] - ff5_full['RF']
    df_ff = df_ff.dropna()
    
    # Perform regression to obtain beta
    ols = sm.OLS(df_ff['Excess return'], sm.add_constant(df_ff[X]), missing='drop').fit(cov_type='HC0')
    
    # Plot results
    if print_results:
        print(ols.summary()) 
    if return_results:
        return ols, df_ff

## Example

In [167]:
capm = ['Mkt-RF']
ff3 = ['Mkt-RF', 'SMB', 'HML']
ff5 = ['Mkt-RF', 'SMB', 'HML', 'CMA', 'RMW']

## 4.1 CAPM

In [188]:
res_capm, df_capm = get_ff_beta(ticker=ticker, X=capm, print_results = True, return_results = True, start=start_date, end=end_date)
beta_capm = res_mkt.params[1]
se_capm = res_mkt.bse[1]

                            OLS Regression Results                            
Dep. Variable:          Excess return   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     152.1
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           1.23e-32
Time:                        06:05:00   Log-Likelihood:                -2430.3
No. Observations:                1007   AIC:                             4865.
Df Residuals:                    1005   BIC:                             4874.
Df Model:                           1                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0259      0.085      0.304      0.7

In [189]:
df_capm[df_capm.columns[0]].values.mean()

0.0983702085402185

## 4.2 Fama French three factors risk model

In [190]:
res_ff3, df_ff3 = get_ff_beta(ticker=ticker, X=ff3, print_results = True, return_results = True, start=start_date, end=end_date)
beta_ff3 = res_ff3.params[1]
se_ff3 = res_ff3.bse[1]

                            OLS Regression Results                            
Dep. Variable:          Excess return   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     60.75
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           4.30e-36
Time:                        06:05:00   Log-Likelihood:                -2419.3
No. Observations:                1007   AIC:                             4847.
Df Residuals:                    1003   BIC:                             4866.
Df Model:                           3                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0232      0.084      0.277      0.7

## 4.3 Fama French five factors risk model

In [191]:
res_ff5, df_ff5 = get_ff_beta(ticker=ticker, X=ff5, print_results = True, return_results = True, start=start_date, end=end_date)
beta_ff5 = res_ff5.params[1]
se_ff5 = res_ff5.bse[1]

                            OLS Regression Results                            
Dep. Variable:          Excess return   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     37.89
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           1.14e-35
Time:                        06:05:01   Log-Likelihood:                -2416.6
No. Observations:                1007   AIC:                             4845.
Df Residuals:                    1001   BIC:                             4875.
Df Model:                           5                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0312      0.084      0.373      0.7

## 4.4 Results summary $\beta$s

In [196]:
## Comparing all $\beta$s

## CAPM predictions
pred_CAPM = res_capm.predict()+df_capm['RF']-res_capm.params[0]
ER_CAPM = np.mean(pred_CAPM)#*252
SE_CAPM = (np.std(pred_CAPM)/math.sqrt(len(pred_CAPM)))*math.sqrt(252)

## FF3 predictions
pred_FF3 = res_ff3.predict()+df_ff3['RF']-res_ff3.params[0]
ER_FF3 = np.mean(pred_FF3)#*252
SE_FF3 = (np.std(pred_FF3)/math.sqrt(len(pred_FF3)))*math.sqrt(252)

## FF5 predictions
pred_FF5 = res_ff5.predict()+df_ff5['RF']-res_ff5.params[0]
ER_FF5 = np.mean(pred_FF5)#*252
SE_FF5 = (np.std(pred_FF5)/math.sqrt(len(pred_FF5)))*math.sqrt(252)

## Results summary
betas = np.array([beta_capm, beta_ff3, beta_ff5])
ses = np.array([se_capm, se_ff3, se_ff5])
predicted_returns = np.array([pred_CAPM, pred_FF3, pred_FF5])
ERs = np.array([ER_CAPM, ER_FF3, ER_FF5])
SEs = np.array([SE_CAPM, SE_FF3, SE_FF5])

## Plot summary
df_summary  = pd.DataFrame({'beta': betas, 'se': ses,'daily ER': ERs, 'SE': SEs})
df_summary.index = np.array(['CAPM', 'FF3', 'FF5'])

df_summary

,beta,se,daily ER,SE
CAPM,1.307,0.106,0.072,0.548
FF3,1.237,0.108,0.075,0.583
FF5,1.153,0.109,0.067,0.591


In [198]:
sample_mean = df_capm[df_capm.columns[0]].values.mean()
sample_se = df_capm[df_capm.columns[0]].values.std()/math.sqrt(len(df_capm))

ER_all = np.array([sample_mean, ER_CAPM, ER_FF3, ER_FF5])
SE_all = np.array([sample_se, SE_CAPM, SE_FF3, SE_FF5])
df_mean = pd.DataFrame({'Daily expected returns': ER_all, 'Std error': STD_all})
df_mean.index = np.array([ticker, 'CAPM', 'FF3', 'FF5'])
df_mean

,Daily expected returns,Std error
TSLA,0.098,0.092
CAPM,0.072,0.548
FF3,0.075,0.583
FF5,0.067,0.591


**Remark** The following features will be added in the finalized version
1. Confidence intervalls
2. Rolling beta option
3. Out of sample to test our strategies